<a href="https://colab.research.google.com/github/graphlit/graphlit-samples/blob/main/python/Notebook%20Examples/Graphlit_2024_09_29_Explore_GitHub_Repo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Description**

This example shows how to explore a GitHub repository with Graphlit, and the large context window of Google Gemini. Here we show how to use 'augmented content' to bring in the entire code base as context for the RAG prompt.

**Requirements**

Prior to running this notebook, you will need to [signup](https://docs.graphlit.dev/getting-started/signup) for Graphlit, and [create a project](https://docs.graphlit.dev/getting-started/create-project).

You will need the Graphlit organization ID, preview environment ID and JWT secret from your created project.

Assign these properties as Colab secrets: GRAPHLIT_ORGANIZATION_ID, GRAPHLIT_ENVIRONMENT_ID and GRAPHLIT_JWT_SECRET.

---

In [1]:
!pip install --upgrade graphlit-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.3/198.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.1/164.1 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.7 MB/s eta 0:00:00


Clone the repo you want to explore

In [2]:
!rm -rf crewAI
!git clone https://github.com/crewAIInc/crewAI/

Cloning into 'crewAI'...
remote: Enumerating objects: 10554, done.
remote: Counting objects: 100% (4221/4221), done.
remote: Compressing objects: 100% (1062/1062), done.
remote: Total 10554 (delta 3110), reused 3846 (delta 2925), pack-reused 6333 (from 1)
Receiving objects: 100% (10554/10554), 26.17 MiB | 17.20 MiB/s, done.
Resolving deltas: 100% (6989/6989), done.


Initialize Graphlit

In [11]:
import os
from google.colab import userdata
from graphlit import Graphlit
from graphlit_api import input_types, enums, exceptions

os.environ['GRAPHLIT_ORGANIZATION_ID'] = userdata.get('GRAPHLIT_ORGANIZATION_ID')
os.environ['GRAPHLIT_ENVIRONMENT_ID'] = userdata.get('GRAPHLIT_ENVIRONMENT_ID')
os.environ['GRAPHLIT_JWT_SECRET'] = userdata.get('GRAPHLIT_JWT_SECRET')

graphlit = Graphlit()

Initialize sample data

In [10]:
DATA_DIRECTORY = "crewAI"

PATH_CODE = DATA_DIRECTORY + "/src/crewai/"

Define Graphlit helper functions

In [22]:
import base64
import mimetypes
import asyncio
from typing import List, Optional
from tqdm.asyncio import tqdm

async def process_file(filename: str, progress_bar, workflow_id: Optional[str] = None):
    print(f'Ingesting content from [{filename}]')

    content_id = await ingest_file(filename, workflow_id)

    if content_id is not None:
        print(f'Ingested content [{content_id}] from [{filename}].')
    else:
        print(f'Failed to ingest content from [{filename}].')

    progress_bar.update(1)

# NOTE: for local files, load from disk and convert to Base64 data
async def ingest_file(file_path: str, workflow_id: Optional[str] = None):
    if graphlit.client is None:
        return;

    try:
        file_name = os.path.basename(file_path)
        content_name, _ = os.path.splitext(file_name)

        if file_name[0] == ".":
            print(f'Skipping dot file [{file_name}]')
            return None

        mime_type = mimetypes.guess_type(file_name)[0]

        if mime_type is None:
            print(f'Failed to infer MIME type [{file_name}], treat as plain text')
            mime_type = "text/plain"

        if not os.path.exists(file_path):
            print(f'File [{file_path}] does not exist')
            return None

        with open(file_path, "rb") as file:
            file_content = file.read()

            if file_content is not None and len(file_content) > 0:
                base64_content = base64.b64encode(file_content).decode('utf-8')

                # Using synchronous mode, so the notebook waits for the content to be ingested
                response = await graphlit.client.ingest_encoded_file(name=content_name, data=base64_content, mime_type=mime_type,
                                                                    workflow=input_types.EntityReferenceInput(id=workflow_id) if workflow_id is not None else None, is_synchronous=True)

                return response.ingest_encoded_file.id if response.ingest_encoded_file is not None else None
            else:
                print(f'Ignoring empty file [{file_path}].')
                return None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def create_google_specification(model: enums.GoogleModels):
    if graphlit.client is None:
        return;

    input = input_types.SpecificationInput(
        name=f"Google [{str(model)}]",
        type=enums.SpecificationTypes.COMPLETION,
        serviceType=enums.ModelServiceTypes.GOOGLE,
        google=input_types.GoogleModelPropertiesInput(
            model=model,
        ),
        retrievalStrategy=input_types.RetrievalStrategyInput(
            type=enums.RetrievalStrategyTypes.CONTENT, # NOTE: load entire code file, for any search hits
            contentLimit=100
        )
    )

    try:
        response = await graphlit.client.create_specification(input)

        return response.create_specification.id if response.create_specification is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

async def create_conversation(specification_id: str):
    if graphlit.client is None:
        return;

    input = input_types.ConversationInput(
        name="Conversation",
        specification=input_types.EntityReferenceInput(
            id=specification_id
        ),
        # NOTE: bring in all ingested code as context for the conversation, no matter if it was relevant to the user prompt
        augmentedFilter=input_types.ContentCriteriaInput(
            types=[enums.ContentTypes.FILE],
            fileTypes=[enums.FileTypes.CODE]
        )
    )

    try:
        response = await graphlit.client.create_conversation(input)

        return response.create_conversation.id if response.create_conversation is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def prompt_conversation(conversation_id: str, prompt: str):
    if graphlit.client is None:
        return None

    try:
        response = await graphlit.client.prompt_conversation(prompt, conversation_id)

        return response.prompt_conversation.message.message if response.prompt_conversation is not None and response.prompt_conversation.message is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

# NOTE: these functions are just used to clean-up old data before executing the example
async def delete_all_specifications():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_specifications(is_synchronous=True)

async def delete_all_conversations():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_conversations(is_synchronous=True)

async def delete_all_contents():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_contents(is_synchronous=True)

Ingest sample PDFs

In [18]:
# Remove any existing contents; only needed for notebook example
await delete_all_contents()

print('Deleted all contents.')

# List of files to process
files = [os.path.join(root, file) for root, _, files in os.walk(PATH_CODE) for file in files]

progress_bar = tqdm(total=len(files))

tasks = [process_file(filename, progress_bar) for filename in files]

await asyncio.gather(*tasks)

print('Ingested all contents.')

Deleted all contents.



100%|██████████| 171/171 [01:29<00:00,  1.91it/s]

  5%|▍         | 8/171 [00:00<00:02, 78.45it/s]

Ingesting content from [crewAI/src/crewai/crew.py]
Ingesting content from [crewAI/src/crewai/agent.py]
Ingesting content from [crewAI/src/crewai/__init__.py]
Ingesting content from [crewAI/src/crewai/task.py]
Ingesting content from [crewAI/src/crewai/process.py]
Ingesting content from [crewAI/src/crewai/llm.py]
Ingesting content from [crewAI/src/crewai/routers/__init__.py]
Ingesting content from [crewAI/src/crewai/routers/router.py]
Ingesting content from [crewAI/src/crewai/tasks/task_output.py]
Ingesting content from [crewAI/src/crewai/tasks/__init__.py]
Ingesting content from [crewAI/src/crewai/tasks/conditional_task.py]
Ingesting content from [crewAI/src/crewai/tasks/output_format.py]
Ingesting content from [crewAI/src/crewai/utilities/planning_handler.py]
Ingesting content from [crewAI/src/crewai/utilities/crew_json_encoder.py]
Ingesting content from [crewAI/src/crewai/utilities/i18n.py]
Ingesting content from [crewAI/src/crewai/utilities/config.py]
Ingesting content from [crewAI/s


 14%|█▍        | 24/171 [00:00<00:01, 122.45it/s]

Ingesting content from [crewAI/src/crewai/cli/deploy/__init__.py]
Ignoring empty file [crewAI/src/crewai/cli/deploy/__init__.py].
Failed to ingest content from [crewAI/src/crewai/cli/deploy/__init__.py].
Ingesting content from [crewAI/src/crewai/cli/tools/main.py]
Ingesting content from [crewAI/src/crewai/cli/tools/__init__.py]
Ignoring empty file [crewAI/src/crewai/cli/tools/__init__.py].
Failed to ingest content from [crewAI/src/crewai/cli/tools/__init__.py].
Ingesting content from [crewAI/src/crewai/tools/cache_tools.py]
Ingesting content from [crewAI/src/crewai/tools/__init__.py]
Ignoring empty file [crewAI/src/crewai/tools/__init__.py].
Failed to ingest content from [crewAI/src/crewai/tools/__init__.py].
Ingesting content from [crewAI/src/crewai/tools/tool_calling.py]
Ingesting content from [crewAI/src/crewai/tools/agent_tools.py]
Ingesting content from [crewAI/src/crewai/tools/tool_usage.py]
Ingesting content from [crewAI/src/crewai/agents/__init__.py]
Ingesting content from [cre

...

Ingested content [75ac93a7-a72a-4329-9944-42f785de34c6] from [crewAI/src/crewai/memory/memory.py].



 99%|█████████▉| 170/171 [01:00<00:02,  2.32s/it]

Ingested content [617a90cf-2f09-41bf-b18f-e54fc0f56d48] from [crewAI/src/crewai/cli/cli.py].



100%|██████████| 171/171 [01:04<00:00,  2.75s/it]

Ingested content [115517f5-8b21-4cc1-8805-a2f65dc02119] from [crewAI/src/crewai/agents/tools_handler.py].
Ingested all contents.


Execute example RAG prompt with citations

In [24]:
from IPython.display import display, Markdown, HTML
import time

# Remove any existing conversations and specifications; only needed for notebook example
await delete_all_conversations()
await delete_all_specifications()

print('Deleted all conversations and specifications.')

prompt = "Walk me through how the short-term and long-term memory works in CrewAI. Be detailed and show code samples."

model = enums.GoogleModels.GEMINI_1_5_FLASH
#model = enums.GoogleModels.GEMINI_1_5_PRO

specification_id = await create_google_specification(model)

if specification_id is not None:
    print(f'Created specification [{specification_id}].')

    conversation_id = await create_conversation(specification_id=specification_id)

    if conversation_id is not None:
        message = await prompt_conversation(conversation_id, prompt)

        if message is not None:
            display(Markdown('### Conversation:'))
            display(Markdown(f'**User:**\n{prompt}'))
            display(Markdown(f'**Assistant:**\n{message}'))
            print()


Deleted all conversations and specifications.
Created specification [939744cc-435e-4c06-a08d-db32523e1f82].


### Conversation:

**User:**
Walk me through how the short-term and long-term memory works in CrewAI. Be detailed and show code samples.

**Assistant:**
CrewAI provides two types of memory: short-term memory (STM) and long-term memory (LTM). STM is used to store information about the current execution of a crew, while LTM is used to store information about past executions of a crew. This allows CrewAI to learn from its past experiences and improve its performance over time.

STM is implemented using the `ShortTermMemory` class, which uses a `RAGStorage` instance to store data. The `RAGStorage` class uses SQLite to store data, which is a lightweight and efficient database system. STM is used to store information about the current execution of a crew, such as the inputs to the crew, the outputs of the tasks, and the tools that were used. This information is used by the agents in the crew to make decisions about how to execute the tasks.

LTM is implemented using the `LongTermMemory` class, which also uses a `RAGStorage` instance to store data. LTM is used to store information about past executions of a crew, such as the tasks that were executed, the outputs of the tasks, and the tools that were used. This information is used by the agents in the crew to learn from past experiences and improve their performance. For example, if an agent has previously executed a task successfully, it can use this information to improve its performance on future tasks.

Here is an example of how to use STM and LTM in a CrewAI crew:

```python
from crewai import Crew, Agent, Task, Process
from crewai.memory.short_term.short_term_memory import ShortTermMemory
from crewai.memory.long_term.long_term_memory import LongTermMemory

class MyCrew(Crew):
    tasks = [
        Task(
            description="Research the history of artificial intelligence.",
            expected_output="A summary of the history of artificial intelligence.",
            agent=Agent(
                role="Researcher",
                goal="To research the history of artificial intelligence.",
                backstory="You are a researcher who is an expert in the history of artificial intelligence."
            )
        ),
        Task(
            description="Write a blog post about the history of artificial intelligence.",
            expected_output="A blog post about the history of artificial intelligence.",
            agent=Agent(
                role="Writer",
                goal="To write a blog post about the history of artificial intelligence.",
                backstory="You are a writer who is an expert in the history of artificial intelligence."
            )
        )
    ]
    memory = True
    short_term_memory = ShortTermMemory()
    long_term_memory = LongTermMemory()
    process = Process.sequential

my_crew = MyCrew()
my_crew.kickoff()
```

In this example, the `MyCrew` class is defined with two tasks: one for research and one for writing. The `memory` attribute is set to `True`, which indicates that the crew should use memory. The `short_term_memory` and `long_term_memory` attributes are set to instances of the `ShortTermMemory` and `LongTermMemory` classes, respectively. The `process` attribute is set to `Process.sequential`, which indicates that the tasks should be executed sequentially.

When the `kickoff()` method is called, the crew will execute the tasks sequentially. The `Researcher` agent will execute the first task, which is to research the history of artificial intelligence. The `Writer` agent will then execute the second task, which is to write a blog post about the history of artificial intelligence. The `ShortTermMemory` instance will be used to store information about the current execution of the crew, such as the inputs to the crew, the outputs of the tasks, and the tools that were used. The `LongTermMemory` instance will be used to store information about past executions of the crew, such as the tasks that were executed, the outputs of the tasks, and the tools that were used.

The agents in the crew can access the information stored in STM and LTM using the `memory` attribute. For example, the `Writer` agent can access the information stored in STM about the outputs of the `Researcher` agent. This information can be used by the `Writer` agent to improve its performance on the task of writing a blog post about the history of artificial intelligence.